## 1. Setup StockX

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))


from ib_insync import *
util.startLoop()
import stock

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=12)
sx  = stock.StockX(ib, 'SABR', intradaySizes=['5 mins', '1 hour'], tradeSizes=['5 mins'])
spy = stock.StockX(ib, 'SPY',  intradaySizes=['5 mins', '1 hour'], tradeSizes=['5 mins'])

## 2. Fundamentals

In [2]:
sx.RUN_FUNDAMENTALS(allowedETFs=['XLY', 'XLK', 'XLC'], maxDaysOld=10)

Using cached data for SABR (age: 2025-03-05 18:19:50)


## 3. Setup

In [3]:
sx.RUN_SETUP(spy, dataType='ohlcv', forceDownload=False, endDate='2025-02-20')

-------------------------------------------------------------------------------------------------
get_hist_data            : SABR 1 day (file_interval='1_day')
Data Path                : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store
Data Stored    - Start   : 2024-01-02 00:00:00, End: 2025-02-20 00:00:00, Rows: 285
Data Requested - Start   : 2024-02-19 00:00:01, End: 2025-02-20
Missing dates            : 1
Getting missing data     : [('2025-02-17', '2025-02-20')]
-------------------------------------------------------------------------------------------------
StockX::setup_frame: SABR 1 day ohlcv 52 weeksAgo 2025-02-20 force_download=False : (252, 5)
-------------------------------------------------------------------------------------------------
get_hist_data            : SABR 5 mins (file_interval='1_min')
Data Path                : C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store
Data Stored    - Start   : 2025-01-27 05:44:00, End: 2025

## 4. PreMarket

In [4]:
sx.RUN_PRE_MARKET('LONG', timeToRun='09:00', minPreMarketScore=50, displayCharts=True, printStats=True) # runs daily and premarket volume analysis

,symbol,status,status_why,score_1D,score_preMkt_vol,score_5M,score_2M,score_AVG,score_best_barsize,pnl_realized,pnl_unrealized,trades_today,win_rate,av_Rratio,last_updated,entry_time,exit_time
0,SABR,PRE_MARKET,Daily TA is set up,28.6,100.0,0.0,0.0,64.3,,0.0,0.0,0,0.0,0.0,2025-03-16 14:26:23.226298,None,None


Score: 1 day             --  28.6
Score: Pre Market Volume --  100.0
Score: preMkt Average    --  64.3


In [5]:
sx.display_columns('1 day')
sx.display_columns('1 day', contains='RTM')

Index(['open', 'high', 'low', 'close', 'volume', 'SPY_close', 'SPY_volume',
       'SPY_low', 'SPY_high', 'SPY_open', 'MA_cl_50', 'MA_cl_150', 'MA_cl_200',
       'ATR_14', 'HP_hi_3', 'LP_lo_3', 'HP_hi_10', 'LP_lo_10', 'Res_1',
       'Res_1_Upper', 'Res_1_Lower', 'Res_2', 'Res_2_Upper', 'Res_2_Lower',
       'Sup_1', 'Sup_1_Upper', 'Sup_1_Lower', 'Sup_2', 'Sup_2_Upper',
       'Sup_2_Lower', 'GapPiv_HP_hi_10', 'L_GPivs', 'L_GRtc', 'L_GPP', 'GapSz',
       'RTM_L_Res_1_Lower', 'RS_SPY_14', 'RS_SPY_14_ma',
       'VAL_L_GapPiv_HP_hi_10_>_1', 'VAL_L_L_GPivs_>_1', 'VAL_L_L_GRtc_>_50',
       'VAL_L_L_GPP_>_80', 'VAL_L_GapSz_<_400', 'VAL_L_RTM_L_Res_1_Lower_>_50',
       'VAL_L_RS_SPY_14_>_1', 'Score_LONG_Daily'],
      dtype='object')

['RTM_L_Res_1_Lower', 'VAL_L_RTM_L_Res_1_Lower_>_50']

## 5. Intraday

In [6]:
sx.RUN_INTRADAY(updateEverySeconds=60, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=True, logTrades=True)

Running LONG on 5 mins
Running LONG on 1 hour
Score: 1 day             --  28.6
Score: Pre Market Volume --  100.0
Score: preMkt Average    --  64.3
----------------------------------------------
Score: 5 mins  --  0.0


In [7]:
import pandas as pd
import strategies.preset_strats as ps
import strategies.signals as sig
import strategies.ta as ta
from typing import Dict, Any, List
from chart.chart import ChartArgs

sx.import_all_market_data(spy)
sx.set_ls('LONG')
sx.setup_TA_1D(lookBack=25, atrSpan=14, sigRow=3, validationRow=4)

if sx.day_score_passed(minScore=50):
    sx.setup_TA_intraday(lookBack=200, atrSpan=50, sigRow=3, validationRow=4)
    
    if sx.pre_market_passed(minScore=50):
        sx.RUN(updateEvery=30, myTradingTimes=[('9:35', '15:00')], maxTrades=5, displayCharts=False, logTrades=True)




spy.frames['1 day'].plot()
sx.frames['1 day'].plot()
sx.frames['1 hour'].plot()
sx.frames['5 mins'].plot()

AttributeError: 'StockX' object has no attribute 'setup_TA_1D'

In [ ]:
sx.display_columns('5 mins', contains='PB')

In [ ]:
dfx = sx.display_columns('5 mins', contains='PB')
dfx.loc['2025-02-20 09:35:00':].head(50)

 # StockX Score

In [9]:
ib.disconnect()

In [ ]:
from dataclasses import dataclass

